In [49]:
import pandas as pd
import ast

from preprocessing_utils import *

csv_path = '../reddit_and_gab/reddit-q.csv'

df = pd.read_csv(csv_path)
df.drop(['id', 'response'], axis=1, inplace=True)
df['hate_speech_idx'] = df['hate_speech_idx'].fillna(0)
# df.head()

In [48]:
new_df = pd.DataFrame(columns=['text', 'is_hate'])

for row_index, row in df.iterrows():
    hate_speech_index = row['hate_speech_idx']
    sent = row['text'].split('\n')
    sent_preprocessed = []

    for s in sent:
        # print("STAVEK:", s)
        s = remove_accented_chars(s)
        s = remove_special_characters(s)
        s = remove_punctuation(s)
        s = s.lower()
        s = expand_contractions(s)
        s = remove_numbers(s)
        s = remove_extra_whitespace_tabs(s)
        s = stemming(s, "ang")
        # print("STAVEK P:", s)
        sent_preprocessed.append(s)


    if hate_speech_index == 0:
        for s in sent_preprocessed:
            if s and s != "remov" and s != "delet":
                new_df.loc[len(new_df.index)] = [s, 0]
    else:
        hate_text_indexes = ast.literal_eval(row['hate_speech_idx'])
        for s_index, s in enumerate(sent_preprocessed, 1):
            if s and s != "remov" and s != "delet":
                if s_index in hate_text_indexes:
                    new_df.loc[len(new_df.index)] = [s, 1]
                else:
                    new_df.loc[len(new_df.index)] = [s, 0]

# new_df.head(100)
new_df.to_csv('../preprocessed_data/reddit_preprocessed.csv')